In [1]:
import requests
from bs4 import BeautifulSoup as bs

import pandas as pd

### Add headers (ethical obligation)

In [2]:
myUa = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0"
myInf = "VaishaliPandiyen PandiyenV@cardiff.ac.uk scraping for uni coursework"
myHeaders = {"User-Agent": f'{myUa}; {myInf}'}
myHeaders

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0; VaishaliPandiyen PandiyenV@cardiff.ac.uk scraping for uni coursework'}

### Run this to update:

In [3]:
aqw_sites = "https://airquality.gov.wales/air-pollution/monitoring-site-summary"
aqw_req_1 = requests.get(aqw_sites, myHeaders)

if aqw_req_1.status_code == 200:
    aqw_soup_1 = bs(aqw_req_1.content)

In [4]:
print("Now scraping:", aqw_soup_1.find('h1', class_ ='page-header')
      .text, "from", aqw_sites)


Now scraping: Monitoring site summary
 from https://airquality.gov.wales/air-pollution/monitoring-site-summary


In [5]:
site_links = aqw_soup_1.find('table', 
                class_='responsive-enabled table table-hover table-striped')
site_links = site_links.find('tbody').find_all('tr')

aqw_places = []
aqw_place_urls = []

for s in site_links:
    place = s.find('td').find('a').text
    aqw_places.append(place)
    
    url_bit = s.find('td').find('a').attrs['href']
    l = f'https://airquality.gov.wales{url_bit}'
    aqw_place_urls.append(l)

    
print(f"Air Quality Wales has air quality monitors in {len(aqw_places)} places")


Air Quality Wales has air quality monitors in 38 places


In [13]:
import datetime

In [113]:
pm25 = 0
no_pm25 = 0
place_list = []
source_list = []
pm25_list = []
pm25_places = []
# el_list = []
# el_subLists = []
# el_dict = {}

for l in aqw_place_urls:
    req = requests.get(l, myHeaders)
    soup = bs(req.content)
    
    all_els=''
    els = soup.find('div', class_='tab-content air-view-pollutant-content').find('table').find('tbody').find_all('tr')
    for el in els:
        e = el.find('td').text
        all_els+=f'{e}, '
        # if e not in el_list:
        #    el_list.append(e)
        #    el_dict[e]=[]     
        
    name = soup.find('section', class_='block block-waq-site-latest-data block-site-latest-data-block clearfix').find('div', class_='field field--name-body field--type-text-with-summary field--label-hidden field--item').find('h2').text
    place_list.append(name)
    
    source_list.append("AQW")
    
    if 'PM2.5' in all_els:
        pm25 += 1
        pm25_list.append('Yes')
        pm25_places.append(name)
        
    else: 
        no_pm25 += 1
        pm25_list.append('No')
        
clean_place_list = [s.strip() for s in place_list]
clean_pm25_places = [s.strip() for s in pm25_places]

version = f'This AQW data is from {datetime.datetime.now()} {datetime.timezone.utc}'
print(version)
print(f'{pm25} places monitor PM2.5 and {no_pm25} do not')
# print(clean_pm25_places)


This AQW data is from 2023-04-27 16:42:42.418843 UTC
17 places monitor PM2.5 and 21 do not
['Caerphilly Fochriw', 'Cardiff Castle Street', 'Cardiff Centre', 'Chepstow A48', 'Narberth', 'Newport St Julians Comp School', 'Port Talbot Dyffryn School', 'Port Talbot Little Warren', 'Port Talbot Margam', 'Port Talbot Prince Street 2', 'Swansea Morriston Roadside', 'Swansea Port Tennant Roadside', 'Swansea Roadside', 'Twynyrodyn', 'Wrexham', 'Wrexham Chirk', 'Wrexham Chirk Community Hospital']


In [71]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [134]:
driver = webdriver.Chrome()

lats = []
lons = []
site_type = []
comments = []

for l in aqw_place_urls:
    driver.get(aqw_place_urls[0])
    # find and click the element that triggers the click event
    wait = WebDriverWait(driver, 5)
    overlay = wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.loadingoverlay")))
    driver.find_element(by=By.LINK_TEXT, value='Site Information').click()
    wait = WebDriverWait(driver, 5)
    dynamic_content = wait.until(EC.presence_of_element_located((By.ID, "siteinformation")))
    # scrape the data from the dynamic content
    data = dynamic_content.find_elements(By.TAG_NAME,'table')
    data2 = data[0].find_element(By.TAG_NAME,'tbody').find_elements(By.TAG_NAME,'tr')
    site = data2[3].find_element(By.TAG_NAME,'td').text
    site_type.append(site)
    coord_txt = data2[3].find_element(By.TAG_NAME,'td').text
    coord = coord_txt.split(',')
    print(coord)
    lat = coord[0]
    lats.append(lat)
#     lon = coord[1]
#     lons.append(lon)
    comment = data2[6].find_element(By.TAG_NAME,'td').text
    comments.append(comment)
    # # close the browser
    driver.quit()
    
print(len(lats), len(lons), len(site_type), len(comments))
            

['52.503850', ' -3.034178']


MaxRetryError: HTTPConnectionPool(host='localhost', port=61240): Max retries exceeded with url: /session/38731bda2f6ed99a7bacbac903e2f641/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002A29C9D65E0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [65]:
d = {'Place': clean_place_list, 'Source': source_list, 'PM 2.5': pm25_list}
df = pd.DataFrame(columns=['Place', 'Source', 'PM 2.5'], data=d)
df


,Place,Source,PM 2.5
0,Aston Hill,AQW,No
1,Bridgend Park Street,AQW,No
2,Caerphilly Blackwood High Street,AQW,No
3,Caerphilly Fochriw,AQW,Yes
4,Caerphilly Islwyn Road Wattsville,AQW,No
5,Caerphilly White Street,AQW,No
6,Cardiff Castle Street,AQW,Yes
7,Cardiff Centre,AQW,Yes
8,Cardiff Newport Road,AQW,No
9,Chepstow A48,AQW,Yes


In [127]:
'''
If we want data on all the monitored polutants: (WIP)

for l in aqw_place_urls: 
        for el in els:
            e = el.find('td').text
        for n in el_dict:
            el_dict[e].append("Yes")
        else:
            el_dict[e].append("No")
            
d = {'Place': clean_place_list, 'Source': source_list}

for e in el_list:
    d[e] = "dummy"
    el_subLists.append([])
    
df = pd.DataFrame(columns=['Place', 'Source', *el_list], data=d)
'''

'\nIf we want data on all the monitored polutants: (WIP)\n\nfor l in aqw_place_urls: \n        for el in els:\n            e = el.find(\'td\').text\n        for n in el_dict:\n            el_dict[e].append("Yes")\n        else:\n            el_dict[e].append("No")\n            \nd = {\'Place\': clean_place_list, \'Source\': source_list}\n\nfor e in el_list:\n    d[e] = "dummy"\n    el_subLists.append([])\n    \ndf = pd.DataFrame(columns=[\'Place\', \'Source\', *el_list], data=d)\n'